In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from gpflow.utilities import print_summary, set_trainable, to_default_float
import gpflow
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

import os
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime

import sys
sys.path.append('../../src')
from data_preparation import get_birth_data, separate_data, train_test_normalise
from useful_fun import plot_sliding_window, number_outof_CI, split_dataframe_by_position


In [5]:
# load data

# whole dataset
data = get_birth_data()
x, y = separate_data(data) # y is unnormalised: good!

# get date for x-axis in plots: mm-yyyy
def get_xticks(df):
    dff = df.groupby(by='m-y').first()
    pos = dff.ids; pos = pos[1:]
    labels = dff.index; labels = labels[1:]
    return pos, labels

# Sliding Window
### Simple one-latent function model with SVGP

        y = f(x) + e
        f = GP(0, RBF)

        l = InvGama()
        sigma = Gamma()

In [6]:
def optimise_model(model, x_train, y_train):
    train_data = (x_train, y_train)
    loss_fn = model.training_loss_closure(train_data) 

    gpflow.utilities.set_trainable(model.q_mu, False)
    gpflow.utilities.set_trainable(model.q_sqrt, False)

    variational_vars = [(model.q_mu, model.q_sqrt)]
    natgrad_opt = gpflow.optimizers.NaturalGradient(gamma=0.1)
    adam_vars = model.trainable_variables
    adam_opt = tf.optimizers.Adam(0.01)

    @tf.function
    def optimisation_step():
        natgrad_opt.minimize(loss_fn, variational_vars)
        adam_opt.minimize(loss_fn, adam_vars)

    epochs = 50
    for epoch in range(1, epochs + 1):
        optimisation_step()

Let's try first train on 10months predict on 2months

In [7]:
splits = 121 # two month split
split_dataframes = split_dataframe_by_position(data, splits)

# evaluation lists
ELBO_train = []
mse_train = []; mae_train = []; n_outof_CI_train = []
mse_test = [];  mae_test = [];  n_outof_CI_test = []

# window sizes
window_size = 5 # 5 for 10months
iterations = splits-window_size

# which iteration to plot
iteration_plot = np.random.randint(low=0, high=iterations)
print('iterations to go through, plot: ', iterations, iteration_plot)

for i in range(splits-window_size):
    # create new dataframe
    df = pd.DataFrame()

    # select data
    for df_ind in range(window_size):
        if df_ind == 0:
            df = split_dataframes[i]
        else :
            df = pd.concat([df, split_dataframes[i+df_ind]], axis=0)

    whole_data = pd.concat([df, split_dataframes[i+window_size]], axis=0)
    pos, labels = get_xticks(whole_data)

    # normalise & separate data
    df_train = df; df_test = split_dataframes[i+window_size]
    df_train, df_test = train_test_normalise(df_train, df_test)
    x_train, y_train = separate_data(df_train)
    x_test, y_test = separate_data(df_test)

    # number of inducing points
    M = int(0.7 * x_train.shape[0])
    
    # build model
    kernel = gpflow.kernels.RBF(lengthscales = 1, variance = 1)
    #Z = x_train.numpy().copy()
    Z = np.linspace(x_train.numpy().min(), x_train.numpy().max(), M)[:, None]
    model = gpflow.models.SVGP(kernel, gpflow.likelihoods.Gaussian(), Z, mean_function=gpflow.mean_functions.Zero(), num_data=M)
    set_trainable(model.likelihood.variance, False)
    model.kernel.lengthscales.prior = tfp.distributions.InverseGamma(to_default_float(0.5), to_default_float(1))
    model.kernel.variance.prior = tfp.distributions.Gamma(to_default_float(2), to_default_float(3))

    # optimise
    optimise_model(model, x_train, y_train)

    # predict
    mean_train, var_train = model.predict_f(x_train)
    mean_test, var_test = model.predict_f(x_test)

    # evaluate
    ELBO_train.append(model.elbo((x_train,y_train)).numpy())
    mse_train.append(mean_squared_error(y_train, mean_train)); mae_train.append(mean_absolute_error(y_train, mean_train)); n_outof_CI_train.append(number_outof_CI(y_train, mean_train, var_train))
    mse_test.append(mean_squared_error(y_test, mean_test)); mae_test.append(mean_absolute_error(y_test, mean_test)); n_outof_CI_test.append(number_outof_CI(y_test, mean_test, var_test))

    if i == iteration_plot:
        plot_sliding_window(x_train, x_test, y_train, y_test, mean_train, mean_test, var_train, var_test, pos, labels, iteration=1)
    

# Average evaluation metrics
print('ELBO Avg: ', np.mean(ELBO_train))
print('Train Avg. MSE, MAE, points outside CI: ', np.mean(mse_train), np.mean(mae_train), np.mean(n_outof_CI_train))
print('Test Avg. MSE, MAE, points outside CI: ', np.mean(mse_test), np.mean(mae_test), np.mean(n_outof_CI_test))

# histogram of CI
fig, ax = plt.subplots(1,2, figsize=(7,10))
ax[0].hist(n_outof_CI_train, bins=50)
ax[0].set_title('Train')
ax[1].hist(n_outof_CI_test, bins=50)
ax[1].set_title('Test')
plt.title('Histogram of number of points that lie outside CI')
plt.show()    

iterations to go through, plot:  116 103


2022-08-12 22:13:30.860846: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


: 

: 

In [ ]:
splits = 60 # 4 month split
split_dataframes = split_dataframe_by_position(data, splits)

# evaluation lists
ELBO_train = []
mse_train = []; mae_train = []; n_outof_CI_train = []
mse_test = [];  mae_test = [];  n_outof_CI_test = []

# window sizes
window_size =  # 5 for 10months
iterations = splits-window_size

# which iteration to plot
iteration_plot = np.random.randint(low=0, high=iterations)
print(iteration_plot)

# number of inducing points
M = None

for i in range(splits-window_size):
    print('iterations to go through: ', iterations)
    df = pd.DataFrame()

    # select data
    for df_ind in range(window_size):
        if df_ind == 0:
            df = split_dataframes[i]
        else :
            df = pd.concat([df, split_dataframes[i+df_ind]], axis=0)

    whole_data = pd.concat([df, split_dataframes[i+window_size]], axis=0)
    pos, labels = get_xticks(whole_data)

    # normalise & separate data
    df_train = df; df_test = split_dataframes[i+window_size]
    df_train, df_test = train_test_normalise(df_train, df_test)
    x_train, y_train = separate_data(df_train)
    x_test, y_test = separate_data(df_test)
    
    # build model
    kernel = gpflow.kernels.RBF(lengthscales = 1, variance = 1)
    Z = x_train.numpy().copy()
    # Z = np.linspace(x_train.numpy().min(), x_train.numpy().max(), M)[:, None]
    model = gpflow.models.SVGP(kernel, gpflow.likelihoods.Gaussian(), Z, mean_function=gpflow.mean_functions.Zero(), num_data=M)
    set_trainable(model.likelihood.variance, False)
    model.kernel.lengthscales.prior = tfp.distributions.InverseGamma(to_default_float(0.5), to_default_float(1))
    model.kernel.variance.prior = tfp.distributions.Gamma(to_default_float(2), to_default_float(3))

    # optimise
    optimise_model(model, x_train, y_train)

    # predict
    mean_train, var_train = model.predict_f(x_train)
    mean_test, var_test = model.predict_f(x_test)

    # evaluate
    ELBO_train.append(model.elbo((x_train,y_train)).numpy())
    mse_train.append(mean_squared_error(y_train, mean_train)); mae_train.append(mean_absolute_error(y_train, mean_train)); n_outof_CI_train.append(number_outof_CI(y_train, mean_train, var_train))
    mse_test.append(mean_squared_error(y_test, mean_test)); mae_test.append(mean_absolute_error(y_test, mean_test)); n_outof_CI_test.append(number_outof_CI(y_test, mean_test, var_test))

    if i == iteration_plot:
        plot_sliding_window(x_train, x_test, y_train, y_test, mean_train, mean_test, var_train, var_test, pos, labels, iteration=1)
    